# Imports

In [1]:
import os
import sys
sys.path.append('..')

import torch

from src.models.yolov8_detector.yolo import load_yolov8_detector
from src.models.sam.sam import load_sam_predictor, segment_images_from_folder

# Data

In [2]:
DATA_DIR = r""

In [3]:
def filter_hidden_folders(folder):
    if folder[0] == '.':
        return False
    return True

In [4]:
folders_list = os.listdir(DATA_DIR)
folders_list = list(filter(filter_hidden_folders, folders_list))
folders_list

['F1_1_2_1.ts-frames_frame-535.png',
 'F1_1_3_1.ts-frames_frame-815.png',
 'F1_1_3_2.ts-frames_frame-1710.png',
 'F1_1_4_1.ts-frames_frame-5.png',
 'F1_1_5_2.ts-frames_frame-450.png']

In [5]:
def count_images(image_folder_path):
    images_num = len(os.listdir(image_folder_path))
    return images_num

def summarize_folders(data_dir, folders_list):
    folders_info = {}
    images_sum = 0
    
    for folder in folders_list:
        images_num = count_images(os.path.join(data_dir, folder))
        folders_info[folder] = images_num
        images_sum += images_num
        
    print(f"There is {images_sum} images:")
    for folder in folders_info.keys():
        print(f"\t{folder} folder has {folders_info[folder]} images")
    

In [6]:
summarize_folders(DATA_DIR, folders_list)

NotADirectoryError: [WinError 267] Неверно задано имя папки: 'D:\\vscode\\ctci\\CTCI\\data\\paper images\\F1_1_2_1.ts-frames_frame-535.png'

# SAM+YOLO Labeling

Yolo detector:

In [5]:
custom_yolo_checkpoint_path = r""
detector = load_yolov8_detector(custom_yolo_checkpoint_path)

SAM:

In [13]:
sam_checkpoint = r""
model_type = "vit_h"
device = "cuda" if torch.cuda.is_available() else "cpu"

predictor = load_sam_predictor(checkpoint_path=sam_checkpoint, model_type=model_type, device=device)

Predictor's device is cpu


## Labeling

In [20]:
folder = folders_list[3]
folder
target_length=1024
narrowing = 0.20
erode_iterations = 1

In [21]:
source_dir = os.path.join(DATA_DIR, folder)
output_dir = os.path.join(DATA_DIR, folder + "_masks")

processes_num = 2

In [23]:
segment_images_from_folder(
    source_dir,
    output_dir,
    detector, predictor,
    target_length=target_length,
    narrowing=narrowing,
    erode_iterations=erode_iterations,
    processes_num=processes_num
)

  0%|          | 0/5 [00:00<?, ?it/s]

Image F1_1_2_1.ts-frames_frame-535.png
0: 608x800 23 bubbles, 267.2ms
Speed: 2.0ms preprocess, 267.2ms inference, 1.0ms postprocess per image at shape (1, 3, 608, 800)


  0%|          | 0/5 [00:01<?, ?it/s]


AssertionError: set_torch_image input must be BCHW with long side 1024.